In [1]:
import torch
import torch.nn as nn
import torch.functional as f
import torch.optim as optim
import numpy as np
from torch.distributions import Normal

In [2]:
%%bash
# Install deps from 
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux

apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip

# Boost libraries
apt-get install libboost-all-dev

# Lua binding dependencies
apt-get install liblua5.1-dev4


Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.4ubuntu1).
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
git is already the newest version (1:2.17.1-1ubuntu0.9).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
unzip is already the newest version (6.0-21ubuntu1.1).
The following additional packages will be installed:
  autoconf automake autopoint autotools-dev debhelper dh-autoreconf
  dh-strip-nondeterminism file freepats gettext gettext-base gir1.2-atk-1.0
  gir1.2-freedesktop gir1.2-gdkpixbuf-2.0 gir1.2-gtk-2.0 gir1.2-ibus-1.0
  gir1.2-pango-1.0 intltool-debian libarchive-cpio-perl libarchive-zip-perl
  libatk1.0-dev libaudio2 libcairo

E: Unable to locate package liblua5.1-dev4
E: Couldn't find any package by glob 'liblua5.1-dev4'
E: Couldn't find any package by regex 'liblua5.1-dev4'


In [3]:
!pip install vizdoom


     |████████████████████████████████| 4.5 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Failed building wheel for vizdoom
Failed to build vizdoom
ERROR: Could not build wheels for vizdoom which use PEP 517 and cannot be installed directly


In [7]:
import numpy as np           # Handle matrices
from vizdoom import *   
from vizdoom import DoomGame     # Doom Environment
import random                # Handling random number generation
import time                  # Handling time calculation
from skimage import transform# Help us to preprocess the frames

from collections import deque# Ordered collection with ends
import matplotlib.pyplot as plt # Display graphs

print ('Done')

ModuleNotFoundError: ignored

In [4]:
!cd /content/drive/MyDrive & git clone https://github.com/mwydmuch/ViZDoom

/bin/bash: line 0: cd: /content/drive/MyDrive: No such file or directory
Cloning into 'ViZDoom'...
remote: Enumerating objects: 15322, done.
remote: Counting objects: 100% (613/613), done.
remote: Compressing objects: 100% (393/393), done.
remote: Total 15322 (delta 354), reused 392 (delta 202), pack-reused 14709
Receiving objects: 100% (15322/15322), 42.90 MiB | 30.38 MiB/s, done.
Resolving deltas: 100% (9659/9659), done.


In [5]:
import torch
import torch.nn as nn
import torch.autograd as autograd 
import torch.nn.functional as F
from torch.distributions import Categorical

class Actor(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(Actor, self).__init__()
        self.input_shape = input_shape
        self.num_actions = num_actions
        
        self.features = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(self.feature_size(), 512),
            nn.ReLU(),
            nn.Linear(512, self.num_actions),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        dist = Categorical(x)
        return dist
    
    def feature_size(self):
        return self.features(autograd.Variable(torch.zeros(1, *self.input_shape))).view(1, -1).size(1)

class Critic(nn.Module):
    def __init__(self, input_shape):
        super(Critic, self).__init__()
        self.input_shape = input_shape
        
        self.features = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(self.feature_size(), 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
    def feature_size(self):
        return self.features(autograd.Variable(torch.zeros(1, *self.input_shape))).view(1, -1).size(1)

In [6]:
class Agent:
  def __init__(self,state_size,action_size):
    self.state_size     = state_size
    self.action_size    = action_size
    self.lr1            = 0.001
    self.lr2            = 0.002
    self.gamma          = 0.99
    self.lamda          = 0.95
    self.n_epochs       = 20
    self.entropy        = 0
    self.batch_size     = 32
    self.clip           = 0.2
    self.entropy        = 0.001
    self.actor          = Actor(self.state_size,self.action_size)
    self.critic         = Critic(self.state_size)
    self.device         = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.actor_optim    = optim.Adam(self.actor.parameters() , lr = self.lr1).to(self.device)
    self.critic_optim   = optim.Adam(self.critic.parameters() , lr = self.lr2).to(self.device)
    self.states         = []
    self.actions        = []
    self.masks          = []
    self.values         = []
    self.rewards        = []
    self.log_probs      = []
  def reset_memory(self):
    self.states         = []
    self.actions        = []
    self.masks          = []
    self.values         = []
    self.rewards        = []
    self.log_probs      = []
  def gaes(self,next_values):
    values = [self.values + next_values]
    gae    = []
    returns= []
    for i in reversed(range(len(self.rewards))):
      delta = self.rewards[i] + self.gamma*self.mask[i]*values[i+1] - values[i]
      gae = delta + self.gamma*self.lamda*gae*self.mask[i]
      returns.insert(0,gae+values[i])
    return returns
  def ppo_iter(self,returns,advantages):
    memory_size = self.states.size(0)
    for _ in range(memory_size // self.batch_size):
        rand_ids = np.random.randint(0, memory_size, self.batch_size)
        yield self.states[rand_ids, :], self.actions[rand_ids], self.log_probs[rand_ids], returns[rand_ids, :], advantages[rand_ids, :]
  def act(self,state):
    s1 = torch.from_numpy(state).unsqueeze(0).to(self.device)
    value = self.critic(s1)
    dist = self.actor(s1)
    action = dist.sample()
    log_probs = dist.log_prob(action)
    return action.item(),log_probs,value
  def step(self,next_state):
    state = torch.from_numpy(next_state)
    next_value = self.critic(state)
    values = torch.cat(self.values).detach()
    logs   = torch.cat(self.log_probs).detach()
    returns = torch.cat(self.gae(self.values)).detach()
    advantages = returns - values
    actions = torch.cat(self.actions)
    for i in range(self.n_epochs):
      for state,action,log_prob,return_,advantage in self.ppo_epochs(returns,advantages):
        dist = self.actor(state)
        entropy = dist.entropy().mean()
        value = self.critic(state)
        new_prob = dist.log_prob(action)
        ratio = (new_prob - log_prob).exp()
        clip = torch.clamp(ratio , 1-self.clip ,1+self.clip)
        s1 = ratio*advantage
        s2 = clip*advantage
        actor_loss = torch.min(s1,s2).mean() - self.entropy*entropy
        critic_loss = (return_ - value).pow(2).mean()
        loss = actor_loss + 0.5* critic_loss
        self.actor_optim.zero_grad()
        self.critic_optim.zero_grad()
        loss.backward()
        self.actor_optim.step()
        self.critic_optim.step()
    self.reset_memory()
  def learn(self,state,action,value,log_prob,reward,done,next_state):
    self.states.append(torch.from_numpy(state).unsqueeze.to(self.device))
    self.actions.append(torch.from_numpy(np.array[action]).to(self.device))
    self.rewards.append(torch.from_numpy(np.array[reward]).to(self.device))
    self.masks.append(torch.from_numpy(np.array[1-done]).to(self.device))
    self.values.append(value)
    self.t_steps = (self.t_steps -1) %100
    if self.t_steps ==0:
      self.learn(next_state)

In [7]:
import numpy as np
import cv2

def preprocess_frame(screen, exclude, output):
    """Preprocess Image.
        
        Params
        ======
            screen (array): RGB Image
            exclude (tuple): Section to be croped (UP, RIGHT, DOWN, LEFT)
            output (int): Size of output image
        """
    # TConver image to gray scale
    screen = cv2.cvtColor(screen, cv2.COLOR_RGB2GRAY)
    
    #Crop screen[Up: Down, Left: right] 
    screen = screen[exclude[0]:exclude[2], exclude[3]:exclude[1]]
    
    # Convert to float, and normalized
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    
    # Resize image to 84 * 84
    screen = cv2.resize(screen, (output, output), interpolation = cv2.INTER_AREA)
    return screen

def stack_frame(stacked_frames, frame, is_new):
    """Stacking Frames.
        
        Params
        ======
            stacked_frames (array): Four Channel Stacked Frame
            frame: Preprocessed Frame to be added
            is_new: Is the state First
        """
    if is_new:
        stacked_frames = np.stack(arrays=[frame, frame, frame, frame])
        stacked_frames = stacked_frames
    else:
        stacked_frames[0] = stacked_frames[1]
        stacked_frames[1] = stacked_frames[2]
        stacked_frames[2] = stacked_frames[3]
        stacked_frames[3] = frame
    
    return stacked_frames

In [8]:
def create_environment():
    game1 = DoomGame()
    
    # Load the correct configuration
    game1.load_config("/content/ViZDoom/scenarios/defend_the_center.cfg")
    
    # Load the correct scenario (in our case defend_the_center scenario)
    game1.set_doom_scenario_path("/content/ViZDoom/scenarios/defend_the_center.wad")
    
    possible_actions  = np.identity(3,dtype=int).tolist()
    
    return game1, possible_actions
game, possible_actions = create_environment()
game.set_window_visible(False)

NameError: ignored

In [ ]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

In [ ]:
def stack_frames(frames, state, is_new=False):
    frame = preprocess_frame(state, (100, -12, -80, 4), 84)
    frames = stack_frame(frames, frame, is_new)

    return frames

In [ ]:
INPUT_SHAPE = (4, 84, 84)
ACTION_SIZE = len(possible_actions)

In [ ]:
agent = Agent(INPUT_SHAPE,ACTION_SIZE)

In [ ]:
start_epoch = 0
scores = []
scores_window = deque(maxlen=20)

In [ ]:
from IPython.display import clear_output

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def train(n_episodes=1000):
    """
    Params
    ======
        n_episodes (int): maximum number of training episodes
    """
    game.init()
    for i_episode in range(start_epoch + 1, n_episodes+1):
        game.new_episode()
        state = stack_frames(None, game.get_state().screen_buffer.transpose(1, 2, 0), True) 
        score = 0
        while True:
            action, log_prob, value = agent.act(state)
            reward = game.make_action(possible_actions[action])
            done = game.is_episode_finished()
            score += reward
            if done:
                agent.step(state, action, value, log_prob, reward, done, state)
                break
            else:
                next_state = stack_frames(state, game.get_state().screen_buffer.transpose(1, 2, 0), False)
                agent.step(state, action, value, log_prob, reward, done, next_state)
                state = next_state
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        
        clear_output(True)
        fig = plt.figure()
        ax = fig.add_subplot(111)
        plt.plot(np.arange(len(scores)), scores)
        plt.ylabel('Score')
        plt.xlabel('Episode #')
        plt.show()
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
    
    return scores

In [ ]:
train(100)

In [10]:
import numpy as np           # Handle matrices
from vizdoom import *   
from vizdoom import DoomGame     # Doom Environment
import random                # Handling random number generation
import time                  # Handling time calculation
from skimage import transform# Help us to preprocess the frames

from collections import deque# Ordered collection with ends
import matplotlib.pyplot as plt # Display graphs

print ('Done')

ModuleNotFoundError: ignored